# Understanding Memory in LLMs

In the previous Notebooks, we successfully explored how OpenAI models can enhance the results from Azure AI Search queries. 

However, we have yet to discover how to engage in a conversation with the LLM. With [Bing Chat](http://chat.bing.com/), for example, this is possible, as it can understand and reference the previous responses.

There is a common misconception that LLMs (Large Language Models) have memory. This is not true. While they possess knowledge, they do not retain information from previous questions asked to them.

In this Notebook, our goal is to illustrate how we can effectively "endow the LLM with memory" by employing prompts and context.

In [1]:
import os
import random
from langchain_community.chat_message_histories import ChatMessageHistory, CosmosDBChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import ConfigurableFieldSpec
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter
from typing import List

from IPython.display import Markdown, HTML, display  

def printmd(string):
    display(Markdown(string))

#custom libraries that we will use later in the app
from common.utils import CustomAzureSearchRetriever, get_answer
from common.prompts import DOCSEARCH_PROMPT

from dotenv import load_dotenv
load_dotenv("credentials.env")

import logging

# Get the root logger
logger = logging.getLogger()
# Set the logging level to a higher level to ignore INFO messages
logger.setLevel(logging.WARNING)

In [2]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]

### Let's start with the basics
Let's use a very simple example to see if the GPT model of Azure OpenAI have memory. We again will be using langchain to simplify our code 

In [5]:
QUESTION = "Tell me some use cases for reinforcement learning"
FOLLOW_UP_QUESTION = "What was my prior question?"

In [6]:
COMPLETION_TOKENS = 1000
# Create an OpenAI instance
llm = AzureChatOpenAI(deployment_name=os.environ["GPT35_DEPLOYMENT_NAME"], 
                      temperature=0.5, max_tokens=COMPLETION_TOKENS)

In [7]:
# We create a very simple prompt template, just the question as is:
output_parser = StrOutputParser()
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an assistant that give thorough responses to users."),
    ("user", "{input}")
])

In [8]:
# Let's see what the GPT model responds
chain = prompt | llm | output_parser
response_to_initial_question = chain.invoke({"input": QUESTION})
display(Markdown(response_to_initial_question))

Reinforcement learning has a wide range of use cases across various industries. Some common use cases for reinforcement learning include:

1. Robotics: Reinforcement learning is used to train robots to perform complex tasks such as grasping objects, navigating through environments, and manipulating objects.

2. Game playing: Reinforcement learning has been successfully applied to train agents to play various games, such as chess, Go, and video games, achieving superhuman performance in some cases.

3. Autonomous vehicles: Reinforcement learning is used to train autonomous vehicles to make decisions in complex and dynamic environments, such as navigating traffic and avoiding obstacles.

4. Finance: Reinforcement learning is used for algorithmic trading, portfolio optimization, and risk management, where agents learn to make optimal decisions in uncertain and dynamic market conditions.

5. Healthcare: Reinforcement learning is applied to optimize treatment plans, drug discovery, and personalized medicine, where agents learn to make decisions based on patient data and medical knowledge.

6. Recommendation systems: Reinforcement learning is used to optimize recommendation algorithms in e-commerce, streaming platforms, and personalized content delivery, where agents learn to maximize user engagement and satisfaction.

7. Energy management: Reinforcement learning is used to optimize energy consumption, demand response, and grid management, where agents learn to make decisions to maximize efficiency and sustainability.

These are just a few examples, and reinforcement learning continues to find applications in diverse fields as the technology matures and advances.

In [9]:
#Now let's ask a follow up question
printmd(chain.invoke({"input": FOLLOW_UP_QUESTION}))

I'm sorry, but as an AI language model, I don't have the capability to recall prior questions. However, if you have a new question or need assistance with something else, feel free to ask and I'll be happy to help!

As you can see, it doesn't remember what it just responded, sometimes it responds based only on the system prompt, or just randomly. This proof that the LLM does NOT have memory and that we need to give the memory as a a conversation history as part of the prompt, like this:

In [10]:
hist_prompt = ChatPromptTemplate.from_template(
"""
    {history}
    Human: {question}
    AI:
"""
)
chain = hist_prompt | llm | output_parser

In [11]:
Conversation_history = """
Human: {question}
AI: {response}
""".format(question=QUESTION, response=response_to_initial_question)

In [12]:
printmd(chain.invoke({"history":Conversation_history, "question": FOLLOW_UP_QUESTION}))

Your prior question was "Tell me some use cases for reinforcement learning"

**Bingo!**, so we now know how to create a chatbot using LLMs, we just need to keep the state/history of the conversation and pass it as context every time

## Now that we understand the concept of memory via adding history as a context, let's go back to our GPT Smart Search engine

From Langchain website:
    
A memory system needs to support two basic actions: reading and writing. Recall that every chain defines some core execution logic that expects certain inputs. Some of these inputs come directly from the user, but some of these inputs can come from memory. A chain will interact with its memory system twice in a given run.

    AFTER receiving the initial user inputs but BEFORE executing the core logic, a chain will READ from its memory system and augment the user inputs.
    AFTER executing the core logic but BEFORE returning the answer, a chain will WRITE the inputs and outputs of the current run to memory, so that they can be referred to in future runs.
    
So this process adds delays to the response, but it is a necessary delay :)

![image](https://python.langchain.com/assets/images/memory_diagram-0627c68230aa438f9b5419064d63cbbc.png)

In [14]:
index1_name = "cogsrch-index-files"
index2_name = "cogsrch-index-books"
indexes = [index1_name, index2_name]

In [15]:
# Initialize our custom retriever 
retriever = CustomAzureSearchRetriever(indexes=indexes, topK=10, reranker_threshold=1)

If you check closely in prompts.py, there is an optional variable in the `DOCSEARCH_PROMPT` called `history`. Now it is the time to use it. It is basically a place holder were we will inject the conversation in the prompt so the LLM is aware of it before it answers.

**Now let's add memory to it:**

In [16]:
store = {} # Our first memory will be a dictionary in memory

# We have to define a custom function that takes a session_id and looks somewhere
# (in this case in a dictionary in memory) for the conversation
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


In [17]:
# We use our original chain with the retriever but removing the StrOutputParser
chain = (
    {
        "context": itemgetter("question") | retriever, 
        "question": itemgetter("question"),
        "history": itemgetter("history")
    }
    | DOCSEARCH_PROMPT
    | llm
)

## Then we pass the above chain to another chain that adds memory to it

output_parser = StrOutputParser()

chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history",
) | output_parser

In [18]:
# This is where we configure the session id
config={"configurable": {"session_id": "abc123"}}

In [20]:
config

{'configurable': {'session_id': 'abc123'}}

Notice below, that we are adding a `history` variable in the call. This variable will hold the chat historywithin the prompt.

In [21]:
printmd(chain_with_history.invoke({"question": QUESTION}, config=config))

Reinforcement learning (RL) has various use cases across different domains. Here are some use cases for reinforcement learning:

1. **Robotics and Autonomous Systems:**
   - Reinforcement learning is used to train robots and autonomous systems to perform complex tasks such as grasping objects, navigating through environments, and learning to perform dexterous operations.

   - *Source:* <sup><a href="https://blobstorage2znp775rdhyvo.blob.core.windows.net/books/Pere_Riche_Pere_Pauvre.pdf" target="_blank">[1]</a></sup>

2. **Game Playing:**
   - Reinforcement learning is widely used in game playing, where agents learn to play games such as chess, Go, and video games. Notably, AlphaGo, developed by DeepMind, is an example of reinforcement learning achieving superhuman performance in the game of Go.

   - *Source:* <sup><a href="https://blobstorage2znp775rdhyvo.blob.core.windows.net/books/Pere_Riche_Pere_Pauvre.pdf" target="_blank">[5]</a></sup>

3. **Recommendation Systems:**
   - Reinforcement learning is applied to develop recommendation systems that learn and optimize user engagement by suggesting relevant content, products, or services.

   - *Source:* <sup><a href="https://blobstorage2znp775rdhyvo.blob.core.windows.net/books/Pere_Riche_Pere_Pauvre.pdf" target="_blank">[6]</a></sup>

4. **Finance and Trading:**
   - Reinforcement learning is used in algorithmic trading to optimize trading strategies and make decisions based on market dynamics, leading to improved financial outcomes.

   - *Source:* <sup><a href="https://blobstorage2znp775rdhyvo.blob.core.windows.net/books/Pere_Riche_Pere_Pauvre.pdf" target="_blank">[7]</a></sup>

5. **Healthcare:**
   - In healthcare, reinforcement learning is used to optimize treatment plans, personalize patient care, and develop strategies for disease management.

   - *Source:* <sup><a href="https://blobstorage2znp775rdhyvo.blob.core.windows.net/books/Pere_Riche_Pere_Pauvre.pdf" target="_blank">[3]</a></sup>

These are just a few examples of the use of reinforcement learning across different domains, showcasing its versatility and potential for solving complex problems.

In [22]:
# Remembers
printmd(chain_with_history.invoke({"question": FOLLOW_UP_QUESTION},config=config))

Your prior question was "Tell me some use cases for reinforcement learning."

In [23]:
# Remembers
printmd(chain_with_history.invoke({"question": "Thank you! Good bye"},config=config))

You're welcome! If you have any more questions in the future, feel free to ask. Goodbye!

## Using CosmosDB as persistent memory

In previous cell we have added local RAM memory to our chatbot. However, it is not persistent, it gets deleted once the app user's session is terminated. It is necessary then to use a Database for persistent storage of each of the bot user conversations, not only for Analytics and Auditing, but also if we wish to provide recommendations in the future. 

Here we will store the conversation history into CosmosDB for future auditing purpose.
We will use a class in LangChain use CosmosDBChatMessageHistory

In [24]:
# Create the function to retrieve the conversation

def get_session_history(session_id: str, user_id: str) -> CosmosDBChatMessageHistory:
    cosmos = CosmosDBChatMessageHistory(
        cosmos_endpoint=os.environ['AZURE_COSMOSDB_ENDPOINT'],
        cosmos_database=os.environ['AZURE_COSMOSDB_NAME'],
        cosmos_container=os.environ['AZURE_COSMOSDB_CONTAINER_NAME'],
        connection_string=os.environ['AZURE_COMOSDB_CONNECTION_STRING'],
        session_id=session_id,
        user_id=user_id
        )

    # prepare the cosmosdb instance
    cosmos.prepare_cosmos()
    return cosmos


In [25]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="user_id",
            annotation=str,
            name="User ID",
            description="Unique identifier for the user.",
            default="",
            is_shared=True,
        ),
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="Unique identifier for the conversation.",
            default="",
            is_shared=True,
        ),
    ],
) | output_parser

In [26]:
# This is where we configure the session id and user id
random_session_id = "session"+ str(random.randint(1, 1000))
ramdom_user_id = "user"+ str(random.randint(1, 1000))

config={"configurable": {"session_id": random_session_id, "user_id": ramdom_user_id}}

In [27]:
config

{'configurable': {'session_id': 'session183', 'user_id': 'user879'}}

In [28]:
printmd(chain_with_history.invoke({"question": QUESTION}, config=config))

Reinforcement learning has numerous real-world applications across various domains. Some of the prominent use cases for reinforcement learning include:

1. **Robotics and Autonomous Systems:** Reinforcement learning is widely used to train robots and autonomous systems to perform complex tasks, such as navigation, object manipulation, and assembly. For instance, robots can be trained to learn how to walk, grasp objects, or perform intricate tasks in unstructured environments, making them more adaptable and versatile<sup><a href="https://blobstorage2znp775rdhyvo.blob.core.windows.net/books/Pere_Riche_Pere_Pauvre.pdf" target="_blank">[1]</a></sup>.

2. **Game Playing:** Reinforcement learning has been successfully applied in playing and mastering complex games. For instance, AlphaGo, an AI program developed by DeepMind, utilized reinforcement learning to become proficient at playing the ancient board game Go, ultimately surpassing human capabilities and defeating world champions<sup><a href="https://blobstorage2znp775rdhyvo.blob.core.windows.net/books/Pere_Riche_Pere_Pauvre.pdf" target="_blank">[6]</a></sup>.

3. **Recommendation Systems:** Reinforcement learning can be used to enhance recommendation systems by learning from user interactions and feedback. This allows the system to continuously improve and provide more accurate and personalized recommendations over time<sup><a href="https://blobstorage2znp775rdhyvo.blob.core.windows.net/books/Pere_Riche_Pere_Pauvre.pdf" target="_blank">[8]</a></sup>.

4. **Finance and Trading:** Reinforcement learning is employed in algorithmic trading to optimize trading strategies and make decisions based on market conditions. It can learn from historical data to identify patterns and trends, ultimately informing trading decisions and portfolio management<sup><a href="https://blobstorage2znp775rdhyvo.blob.core.windows.net/books/Pere_Riche_Pere_Pauvre.pdf" target="_blank">[11]</a></sup>.

5. **Healthcare:** Reinforcement learning is utilized in healthcare for personalized treatment planning, drug discovery, and optimizing clinical trials. It can help in developing adaptive treatment plans and understanding patient responses to different interventions, ultimately leading to more effective and tailored healthcare solutions<sup><a href="https://blobstorage2znp775rdhyvo.blob.core.windows.net/books/Pere_Riche_Pere_Pauvre.pdf" target="_blank">[14]</a></sup>.

6. **Supply Chain Management:** Reinforcement learning can be applied to optimize logistics and supply chain operations. It can help in route optimization, inventory management, and demand forecasting, leading to more efficient and cost-effective supply chain processes<sup><a href="https://blobstorage2znp775rdhyvo.blob.core.windows.net/books/Pere_Riche_Pere_Pauvre.pdf" target="_blank">[17]</a></sup>.

These use cases demonstrate the versatility and potential of reinforcement learning in addressing complex problems across different industries and domains. It offers a powerful framework for enabling machines to learn and make intelligent decisions in dynamic and uncertain environments.

In [29]:
# Remembers
printmd(chain_with_history.invoke({"question": FOLLOW_UP_QUESTION},config=config))

Your prior question was, "Tell me some use cases for reinforcement learning."

In [30]:
# Remembers
printmd(chain_with_history.invoke(
    {"question": "Can you tell me a one line summary of our conversation?"},
    config=config))

We discussed various real-world applications of reinforcement learning across domains such as robotics, game playing, recommendation systems, finance, healthcare, and supply chain management.

In [31]:
try:
    printmd(chain_with_history.invoke(
    {"question": "Thank you very much!"},
    config=config))
except Exception as e:
    print(e)

You're very welcome! If you have any more questions in the future, feel free to ask.

In [32]:
printmd(chain_with_history.invoke(
    {"question": "I do have one more question, why did you give me a one line summary?"},
    config=config))

I provided a one-line summary to concisely capture the main topic and key points of our conversation for a quick and clear reference.

In [33]:
printmd(chain_with_history.invoke(
    {"question": "why not 2?"},
    config=config))

I understand. I should have provided a more detailed summary of our conversation, and I appreciate your feedback. Thank you for pointing that out.

#### Let's check our Azure CosmosDB to see the whole conversation


![CosmosDB Memory](./images/cosmos-chathistory.png)

# Summary
##### Adding memory to our application allows the user to have a conversation, however this feature is not something that comes with the LLM, but instead, memory is something that we must provide to the LLM in form of context of the question.

We added persitent memory using CosmosDB.

We also can notice that the current chain that we are using is smart, but not that much. Although we have given memory to it, it searches for similar docs everytime, regardless of the input. This doesn't seem efficient, but regardless, we are very close to finish our first RAG-talk to your data Bot.


## <u>Important Note</u>:<br>
As we proceed, while all the code will remain compatible with GPT-3.5 models (1106 or newer), we highly recommend transitioning to GPT-4. Here's why:

**GPT-3.5-Turbo** can be likened to a 7-year-old child. You can provide it with concise instructions, but it struggles sometimes to follow them accurately (not too reliable). Additionally, its limited "memory" (token context) can make sustained conversations challenging. Its response are also simple not deep.

**GPT-4-Turbo** exhibits the capabilities of a 10-12-year-old child. It possesses enhanced reasoning skills, consistently adheres to instructions and its answers are beter. It has extended memory retention (larger context size) for instructions, and it excels at following them. Its responses are deep and thorough.


# NEXT
We know now how to do a Smart Search Engine that can power a chatbot!! great!

In the next notebook 6, we are going to build our first RAG bot. In order to do this we will introduce the concept of Agents.